# 🚦 Smart City Traffic Signal Optimization using CSP

## 📌 Overview
`TrafficSystem` simulates a **4×5 smart city intersection grid**, optimizing traffic flow using a **Constraint Satisfaction Problem (CSP)** approach. It manages dynamic real-time traffic (cars, buses, emergency vehicles) while considering:
- Hard constraints (conflict-free signals, emergency routes, pedestrian timing)
- Soft constraints (fuel efficiency, district fairness, congestion reduction)

---

## 🧱 Class: `Intersection`
Represents a single adaptive traffic light node.

### 🔑 Key Attributes
- `id`, `row`, `col`, `district`: Position & ID
- `light_states`: `[N, E, S, W]` (0: red, 1: green, 2: yellow)
- `green_durations`, `red_durations`, `yellow_duration`
- `queues`, `wait_cycles`, `congestion_levels`
- `pedestrian_waiting`, `pedestrian_timer`
- `road_closures`: `[bool × 4]` for direction blocks

### 🔧 Methods
- `update(dt, simulation_time)`: Manages light state transitions
- `set_light(direction, state, duration)`: Controls light timing and synchronization
- `draw(screen)`: Renders the intersection

---

## 🚗 Class: `Vehicle`
Represents individual vehicles in the grid.

### 🔑 Key Attributes
- `type`: `'car'`, `'bus'`, `'emergency'`
- `start_intersection`, `end_intersection`, `path`, `speed`
- `fuel_consumption`, `wait_time`, `behind_schedule`
- `color`: Visual representation by type

### 🔧 Methods
- `update(...)`: Handles movement, light-checking, and delays
- `draw(screen)`: Visualizes the vehicle

---

## 🧠 Class: `TrafficSystem`
Main simulation controller and CSP engine.

### 🔑 Key Attributes
- `intersections`, `vehicles`, `buses`, `emergency_vehicles`
- `simulation_time`, `weather`, `time_of_day`
- `green_wave_paths`, `vehicle_chains`, `road_closures`
- `csp_variables`, `csp_domains`: CSP modeling structures
- `stats`: Tracks delays, fuel use, fairness, pedestrian timing

### 🧮 CSP Optimization
Runs every 5 minutes with:
- **Hard Constraints**:
  - No conflicting green lights
  - Emergency vehicle preference
  - Bus schedule adherence
  - Max wait cycles
  - Pedestrian crossing window
- **Soft Constraints**:
  - Congestion balancing
  - Fuel efficiency
  - District fairness
  - Green waves for vehicle chains

### ⚙️ Methods
- `initialize_random_traffic()`: Adds vehicles and road closures
- `optimize_traffic_signals()`: Applies CSP to set optimal signal timings
- `update(dt)`: Updates simulation entities and metrics
- `draw()`: Renders city map, stats, and UI
- `run()`: Main simulation loop with key controls

---

## ⏱️ Time Complexity
- **CSP Variables**: 80 (4 lights × 20 intersections)
- **Domain Size**: ~10 (timing values per light)
- **Worst-case CSP**: `O(10^80)` (backtracking)
- **Heuristic CSP**: `O(80²)` = `O(6400)` (conflict resolution heuristic)

---

## 🚀 Usage & Controls
- `Run the script` to start simulation
- Controls:
  - `Space`: Pause/resume
  - `C`: Add car
  - `B`: Add bus
  - `E`: Add emergency vehicle
  - `R`: Add road closure
  - `W`: Cycle weather (clear → rain → snow)
  - `T`: Cycle time of day (night → normal → rush)

---


In [1]:
import pygame
import numpy as np
import random
import math
# import time
from collections import deque, defaultdict
import heapq

# Initialize pygame
pygame.init()

# Constants
SCREEN_WIDTH = 1350
SCREEN_HEIGHT = 700
GRID_SIZE = 100
GRID_ROWS = 4
GRID_COLS = 5
INTERSECTION_RADIUS = 10
VEHICLE_SIZE = 8
LIGHT_SIZE = 5
ANIMATION_SPEED = 10  # Higher is faster
PEDESTRIAN_INTERVAL = 60  # Pedestrian crossings must be available every n seconds

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
YELLOW = (255, 255, 0)
BLUE = (0, 0, 255)
GRAY = (200, 200, 200)
DARK_GRAY = (100, 100, 100)
PURPLE = (128, 0, 128)
ORANGE = (255, 165, 0)
LIGHT_BLUE = (173, 216, 230)

class Intersection:
    def __init__(self, id, row, col, district):
        self.id = id
        self.row = row
        self.col = col
        self.district = district  # For fairness across districts
        self.x = col * GRID_SIZE + GRID_SIZE // 2
        self.y = row * GRID_SIZE + GRID_SIZE // 2
        
        # Traffic light states (0: red, 1: green, 2: yellow)
        # Order: North, East, South, West
        self.light_states = [0, 0, 0, 0]
        
        # Traffic light timers (seconds remaining in current state)
        self.light_timers = [0, 0, 0, 0]
        
        # Light durations (can be adjusted by CSP)
        self.green_durations = [30, 30, 30, 30]  # 15-60s range
        self.red_durations = [60, 60, 60, 60]    # 30-90s range
        self.yellow_duration = 5
        
        # Queue lengths in each direction
        self.queues = [0, 0, 0, 0]
        
        # Wait cycles for each direction
        self.wait_cycles = [0, 0, 0, 0]
        
        # Pedestrian waiting flags and timers
        self.pedestrian_waiting = [False, False, False, False]
        self.pedestrian_timer = 0
        self.last_pedestrian_crossing = 0
        
        # Priority flags for roads (used by CSP)
        self.road_priorities = [1, 1, 1, 1]  # Higher values = higher priority
        
        # Congestion levels
        self.congestion_levels = [0, 0, 0, 0]
        
        # Fairness metrics
        self.green_time_history = [0, 0, 0, 0]  # Total green time given to each direction
        
        # Construction/road closure flags
        self.road_closures = [False, False, False, False]
        
        # Initialize with random light state (N-S or E-W)
        if random.random() > 0.5:
            # N-S green
            self.light_states[0] = 1
            self.light_states[2] = 1
            self.light_timers[0] = self.green_durations[0]
            self.light_timers[2] = self.green_durations[2]
        else:
            # E-W green
            self.light_states[1] = 1
            self.light_states[3] = 1
            self.light_timers[1] = self.green_durations[1]
            self.light_timers[3] = self.green_durations[3]
    
    def update(self, dt, simulation_time):
        # Update light timers
        for i in range(4):
            if self.light_timers[i] > 0:
                self.light_timers[i] -= dt
                
                # If timer expires, change light state
                if self.light_timers[i] <= 0:
                    if self.light_states[i] == 1:  # Green to Yellow
                        self.light_states[i] = 2
                        self.light_timers[i] = self.yellow_duration
                        # Update green time history for fairness metrics
                        self.green_time_history[i] += self.green_durations[i]
                    elif self.light_states[i] == 2:  # Yellow to Red
                        self.light_states[i] = 0
                        self.light_timers[i] = self.red_durations[i]
                        # Increment wait cycle for this direction
                        self.wait_cycles[i] += 1
                    elif self.light_states[i] == 0:  # Red to Green
                        self.light_states[i] = 1
                        self.light_timers[i] = self.green_durations[i]
                        # Reset wait cycle for this direction
                        self.wait_cycles[i] = 0
        
        # Update pedestrian timer
        if self.pedestrian_timer > 0:
            self.pedestrian_timer -= dt
            if self.pedestrian_timer <= 0:
                # Reset pedestrian waiting flags
                self.pedestrian_waiting = [False, False, False, False]
                self.last_pedestrian_crossing = simulation_time
    
    def set_light(self, direction, state, duration):
        self.light_states[direction] = state
        self.light_timers[direction] = duration
        
        # If setting to green, update opposite direction too (for N-S or E-W coordination)
        if state == 1:
            opposite_dir = (direction + 2) % 4
            self.light_states[opposite_dir] = state
            self.light_timers[opposite_dir] = duration
    
    def draw(self, screen):
        # Draw intersection
        pygame.draw.circle(screen, BLACK, (self.x, self.y), INTERSECTION_RADIUS)
        
        # Draw intersection ID
        font = pygame.font.SysFont(None, 24)
        text = font.render(f"I{self.id}", True, BLACK)
        screen.blit(text, (self.x - 10, self.y - 30))
        
        # Draw district indicator (for fairness visualization)
        district_colors = [LIGHT_BLUE, (255, 200, 200), (200, 255, 200), (255, 255, 200)]
        if self.district < len(district_colors):
            pygame.draw.circle(screen, district_colors[self.district], 
                              (self.x + 20, self.y - 20), 5)
        
        # Draw traffic lights
        light_positions = [
            (self.x, self.y - 20),  # North
            (self.x + 20, self.y),  # East
            (self.x, self.y + 20),  # South
            (self.x - 20, self.y)   # West
        ]
        
        for i, pos in enumerate(light_positions):
            if self.road_closures[i]:
                # Draw X for closed roads
                pygame.draw.line(screen, BLACK, 
                                (pos[0]-5, pos[1]-5), 
                                (pos[0]+5, pos[1]+5), 2)
                pygame.draw.line(screen, BLACK, 
                                (pos[0]-5, pos[1]+5), 
                                (pos[0]+5, pos[1]-5), 2)
            else:
                if self.light_states[i] == 0:
                    color = RED
                elif self.light_states[i] == 1:
                    color = GREEN
                else:
                    color = YELLOW
                pygame.draw.circle(screen, color, pos, LIGHT_SIZE)
        
        # Draw pedestrians if waiting
        for i, waiting in enumerate(self.pedestrian_waiting):
            if waiting:
                ped_positions = [
                    (self.x - 15, self.y - 15),  # North
                    (self.x + 15, self.y - 15),  # East
                    (self.x + 15, self.y + 15),  # South
                    (self.x - 15, self.y + 15)   # West
                ]
                pygame.draw.circle(screen, PURPLE, ped_positions[i], 4)
        
        # Draw queue lengths (optional)
        for i, queue in enumerate(self.queues):
            if queue > 0:
                queue_positions = [
                    (self.x - 25, self.y - 25),  # North
                    (self.x + 25, self.y - 25),  # East
                    (self.x + 25, self.y + 25),  # South
                    (self.x - 25, self.y + 25)   # West
                ]
                text = font.render(str(queue), True, BLACK)
                screen.blit(text, queue_positions[i])

class Vehicle:
    def __init__(self, id, type, start_intersection, end_intersection, path=None, schedule=None):
        self.id = id
        self.type = type  # 'car', 'bus', 'emergency'
        self.start_intersection = start_intersection
        self.end_intersection = end_intersection
        self.current_intersection = start_intersection
        self.next_intersection = None
        self.path = path if path else []
        self.path_index = 0
        self.x = start_intersection.x
        self.y = start_intersection.y
        self.speed = 2 if type == 'emergency' else 1.5 if type == 'bus' else 1
        self.wait_time = 0
        self.total_wait_time = 0
        self.direction = None  # 0: North, 1: East, 2: South, 3: West
        self.target_x = self.x
        self.target_y = self.y
        self.moving = False
        self.arrived = False
        self.wait_cycles = 0
        self.fuel_consumption = 0  # For tracking fuel efficiency
        self.schedule = schedule  # For buses: list of [intersection_id, expected_time]
        self.schedule_index = 0
        self.behind_schedule = False
        
        # Set color based on type
        if self.type == 'car':
            self.color = BLUE
        elif self.type == 'bus':
            self.color = ORANGE
        else:  # emergency
            self.color = RED
    
    def update(self, dt, intersections, simulation_time, weather_condition):
        if self.arrived:
            return
            
        # Apply weather effects to speed
        effective_speed = self.speed
        if weather_condition == 1:  # Rain
            effective_speed *= 0.7  # 30% slower in rain
        elif weather_condition == 2:  # Snow
            effective_speed *= 0.5  # 50% slower in snow
            
        if not self.moving and self.path_index < len(self.path) - 1:
            # Get current and next intersection
            current_id = self.path[self.path_index]
            next_id = self.path[self.path_index + 1]
            
            current = next((i for i in intersections if i.id == current_id), None)
            next_intersection = next((i for i in intersections if i.id == next_id), None)
            
            if current and next_intersection:
                self.current_intersection = current
                self.next_intersection = next_intersection
                
                # Determine direction
                if next_intersection.row < current.row:
                    self.direction = 0  # North
                elif next_intersection.col > current.col:
                    self.direction = 1  # East
                elif next_intersection.row > current.row:
                    self.direction = 2  # South
                else:
                    self.direction = 3  # West
                
                # Check if road is closed
                if current.road_closures[self.direction]:
                    # Road is closed, recalculate path
                    return
                
                # Check if light is green or if it's an emergency vehicle
                can_move = (self.type == 'emergency' or 
                           current.light_states[self.direction] == 1)
                
                if can_move:
                    # Start moving to next intersection
                    self.moving = True
                    self.target_x = next_intersection.x
                    self.target_y = next_intersection.y
                    self.wait_time = 0
                    self.wait_cycles = 0
                else:
                    # Wait at red light
                    self.wait_time += dt
                    self.total_wait_time += dt
                    
                    # Check if bus is behind schedule
                    if self.type == 'bus' and self.schedule and self.schedule_index < len(self.schedule):
                        expected_time = self.schedule[self.schedule_index][1]
                        if simulation_time > expected_time:
                            self.behind_schedule = True
                    
                    # Increment fuel consumption during idle
                    self.fuel_consumption += dt * 0.1  # Idle fuel consumption
        
        if self.moving:
            # Move towards target
            dx = self.target_x - self.x
            dy = self.target_y - self.y
            distance = math.sqrt(dx*dx + dy*dy)
            
            if distance < effective_speed * dt:
                # Arrived at next intersection
                self.x = self.target_x
                self.y = self.target_y
                self.moving = False
                self.path_index += 1
                
                # Update bus schedule
                if self.type == 'bus' and self.schedule and self.schedule_index < len(self.schedule):
                    if self.path[self.path_index] == self.schedule[self.schedule_index][0]:
                        self.schedule_index += 1
                        self.behind_schedule = False
                
                # Check if reached destination
                if self.path_index >= len(self.path) - 1:
                    self.arrived = True
            else:
                # Move towards target
                self.x += dx / distance * effective_speed * dt
                self.y += dy / distance * effective_speed * dt
                
                # Increment fuel consumption during movement
                self.fuel_consumption += dt * 0.2  # Moving fuel consumption
                
                # Extra fuel for stop-and-go (if recently waited)
                if self.wait_time > 0:
                    self.fuel_consumption += 0.5  # Penalty for stop-and-go
    
    def draw(self, screen):
        if self.arrived:
            return
        
        # Draw vehicle
        if self.type == 'emergency':
            # Draw as a star for emergency vehicles
            points = []
            for i in range(5):
                angle = math.pi/2 + i * 2*math.pi/5
                points.append((self.x + VEHICLE_SIZE * math.cos(angle),
                              self.y + VEHICLE_SIZE * math.sin(angle)))
                angle += math.pi/5
                points.append((self.x + VEHICLE_SIZE/2 * math.cos(angle),
                              self.y + VEHICLE_SIZE/2 * math.sin(angle)))
            pygame.draw.polygon(screen, self.color, points)
        elif self.type == 'bus':
            # Draw as a rectangle for buses
            pygame.draw.rect(screen, self.color, 
                            (self.x - VEHICLE_SIZE, self.y - VEHICLE_SIZE/2, 
                             VEHICLE_SIZE*2, VEHICLE_SIZE))
            
            # Indicate if behind schedule
            if self.behind_schedule:
                pygame.draw.circle(screen, RED, (int(self.x), int(self.y - VEHICLE_SIZE/2 - 5)), 3)
        else:
            # Draw as a circle for cars
            pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), VEHICLE_SIZE//2)
        
        # Show waiting vehicles differently (add a red outline if waiting at red light)
        if not self.moving and self.wait_time > 0 and self.type != 'emergency':
            pygame.draw.circle(screen, RED, (int(self.x), int(self.y)), VEHICLE_SIZE//2 + 2, 1)
            
            # Show wait cycles for vehicles waiting too long
            if self.wait_cycles >= 3:
                font = pygame.font.SysFont(None, 16)
                text = font.render("!", True, RED)
                screen.blit(text, (self.x - 2, self.y - 12))

class TrafficSystem:
    def __init__(self):
        # Create screen
        self.screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
        pygame.display.set_caption("Smart City Traffic Optimization")
        
        # Create clock
        self.clock = pygame.time.Clock()
        
        # Create districts (for fairness)
        self.districts = [
            [1, 1, 2, 2, 2],
            [1, 1, 2, 2, 2],
            [3, 3, 4, 4, 4],
            [3, 3, 4, 4, 4]
        ]
        
        # Create intersections
        self.intersections = []
        for i in range(GRID_ROWS):
            for j in range(GRID_COLS):
                id = i * GRID_COLS + j + 1
                district = self.districts[i][j]
                self.intersections.append(Intersection(id, i, j, district))
        
        # Create vehicles
        self.vehicles = []
        
        # Create emergency vehicles
        self.emergency_vehicles = []
        
        # Create buses
        self.buses = []
        
        # Simulation time
        self.simulation_time = 0
        self.last_optimization_time = 0
        
        # Statistics
        self.stats = {
            'average_wait_time': 0,
            'congestion_level': 0,
            'emergency_response_time': 0,
            'pedestrian_wait_time': 0,
            'fuel_consumption': 0,
            'fairness_index': 100  # 0-100, higher is better
        }
        
        # Weather condition (0: clear, 1: rain, 2: snow)
        self.weather = 0
        
        # Time of day (0: night, 1: normal, 2: rush hour)
        self.time_of_day = 1
        
        # Green wave paths
        self.green_wave_paths = []
        
        # Vehicle chains (for green wave preference)
        self.vehicle_chains = []
        
        # Road closures (for construction)
        self.road_closures = []
        
        # CSP variables and domains
        self.csp_variables = []
        self.csp_domains = {
            'green_duration': list(range(15, 61, 5)),  # 15-60s in 5s increments
            'red_duration': list(range(30, 91, 5)),    # 30-90s in 5s increments
            'priority': list(range(1, 11))             # 1-10 priority levels
        }
        
        # Initialize random traffic
        self.initialize_random_traffic()
        
        # Initialize CSP variables
        self.initialize_csp_variables()
    
    def initialize_random_traffic(self):
        # Add some random vehicles
        for _ in range(20):
            self.add_random_vehicle('car')
        
        # Add a few buses with schedules
        for _ in range(3):
            self.add_bus_with_schedule()
        
        # Add an emergency vehicle
        start = random.choice(self.intersections)
        end = random.choice([i for i in self.intersections if i.id != start.id])
        self.add_emergency_vehicle(start.id, end.id)
        
        # Add random road closures (construction)
        self.add_random_road_closure()
    
    def initialize_csp_variables(self):
        """Initialize CSP variables for each intersection"""
        self.csp_variables = []
        for intersection in self.intersections:
            for direction in range(4):
                self.csp_variables.append({
                    'intersection_id': intersection.id,
                    'direction': direction,
                    'green_duration': intersection.green_durations[direction],
                    'red_duration': intersection.red_durations[direction],
                    'priority': intersection.road_priorities[direction]
                })
    
    def add_random_vehicle(self, type):
        start = random.choice(self.intersections)
        end = random.choice([i for i in self.intersections if i.id != start.id])
        path = self.find_path(start.id, end.id)
        
        if path:
            vehicle = Vehicle(len(self.vehicles), type, start, end, path)
            self.vehicles.append(vehicle)
            
            if type == 'bus':
                self.buses.append(vehicle)
            
            return vehicle
        return None
    
    def add_bus_with_schedule(self):
        """Add a bus with a predefined schedule"""
        # Create a random path through the city
        start = random.choice(self.intersections)
        path = [start.id]
        
        # Add 3-5 stops
        num_stops = random.randint(3, 5)
        current = start
        
        for _ in range(num_stops):
            # Find a neighbor
            neighbors = self.get_neighbors(current.id)
            if not neighbors:
                break
                
            next_id = random.choice(neighbors)
            next_intersection = next((i for i in self.intersections if i.id == next_id), None)
            
            if next_intersection and next_id not in path:
                path.append(next_id)
                current = next_intersection
        
        # Add end point
        end = next((i for i in self.intersections if i.id == path[-1]), None)
        
        if len(path) >= 2 and start and end:
            # Create schedule (expected arrival times)
            schedule = []
            expected_time = 0
            
            for i, intersection_id in enumerate(path):
                if i > 0:
                    # Estimate travel time between stops
                    expected_time += 30  # 30 seconds between stops
                schedule.append([intersection_id, expected_time])
            
            # Create bus
            vehicle = Vehicle(len(self.vehicles), 'bus', start, end, path, schedule)
            self.vehicles.append(vehicle)
            self.buses.append(vehicle)
            
            return vehicle
        return None
    
    def add_emergency_vehicle(self, start_id, end_id):
        """Add an emergency vehicle with a specific route"""
        start = next((i for i in self.intersections if i.id == start_id), None)
        end = next((i for i in self.intersections if i.id == end_id), None)
        
        if start and end:
            path = self.find_path(start_id, end_id)
            
            if path:
                vehicle = Vehicle(len(self.vehicles), 'emergency', start, end, path)
                self.vehicles.append(vehicle)
                self.emergency_vehicles.append(vehicle)
                
                # Create a green wave path
                self.green_wave_paths.append(path)
                
                return vehicle
        return None
    
    def add_random_road_closure(self):
        """Add a random road closure (construction)"""
        intersection = random.choice(self.intersections)
        direction = random.randint(0, 3)
        
        # Close the road in this direction
        intersection.road_closures[direction] = True
        
        # Add to list of closures
        self.road_closures.append((intersection.id, direction))
    
    def get_neighbors(self, intersection_id):
        """Get neighboring intersections"""
        intersection = next((i for i in self.intersections if i.id == intersection_id), None)
        if not intersection:
            return []
            
        neighbors = []
        row, col = intersection.row, intersection.col
        
        # Check all four directions
        directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]  # N, E, S, W
        
        for i, (dr, dc) in enumerate(directions):
            new_row, new_col = row + dr, col + dc
            
            if 0 <= new_row < GRID_ROWS and 0 <= new_col < GRID_COLS:
                # Check if road is closed
                if not intersection.road_closures[i]:
                    for neighbor in self.intersections:
                        if neighbor.row == new_row and neighbor.col == new_col:
                            neighbors.append(neighbor.id)
        
        return neighbors
    
    def find_path(self, start_id, end_id):
        """Find shortest path between two intersections using A*"""
        def heuristic(a, b):
            a_intersection = next((i for i in self.intersections if i.id == a), None)
            b_intersection = next((i for i in self.intersections if i.id == b), None)
            
            if not a_intersection or not b_intersection:
                return 0
                
            return abs(a_intersection.row - b_intersection.row) + abs(a_intersection.col - b_intersection.col)
        
        # Initialize open and closed sets
        open_set = [(0, start_id)]
        closed_set = set()
        came_from = {}
        g_score = {start_id: 0}
        f_score = {start_id: heuristic(start_id, end_id)}
        
        while open_set:
            _, current = heapq.heappop(open_set)
            
            if current == end_id:
                # Reconstruct path
                path = [current]
                while current in came_from:
                    current = came_from[current]
                    path.append(current)
                return path[::-1]
            
            closed_set.add(current)
            
            # Get neighbors
            neighbors = self.get_neighbors(current)
            
            for neighbor in neighbors:
                if neighbor in closed_set:
                    continue
                
                tentative_g = g_score[current] + 1
                
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + heuristic(neighbor, end_id)
                    
                    # Add to open set if not already there
                    if not any(neighbor == id for _, id in open_set):
                        heapq.heappush(open_set, (f_score[neighbor], neighbor))
        
        return []  # No path found
    
    def identify_vehicle_chains(self):
        """Identify chains of vehicles for green wave preference"""
        # Group vehicles by road segments
        road_segments = defaultdict(list)
        
        for vehicle in self.vehicles:
            if not vehicle.arrived and vehicle.path_index < len(vehicle.path) - 1:
                current = vehicle.path[vehicle.path_index]
                next_id = vehicle.path[vehicle.path_index + 1]
                segment = (current, next_id)
                road_segments[segment].append(vehicle)
        
        # Identify chains (segments with multiple vehicles)
        self.vehicle_chains = []
        
        for segment, vehicles in road_segments.items():
            if len(vehicles) >= 3:  # At least 3 vehicles to form a chain
                self.vehicle_chains.append({
                    'segment': segment,
                    'vehicles': vehicles,
                    'count': len(vehicles)
                })
    
    def optimize_traffic_signals(self):
        """Optimize traffic signals using CSP approach"""
        print(f"Running CSP optimization at time {self.simulation_time:.1f}")
        
        # Visual indicator that optimization is running
        self.optimization_indicator = True
        self.optimization_start_time = self.simulation_time
        
        # Update CSP variables
        self.update_csp_variables()
        
        # Identify vehicle chains
        self.identify_vehicle_chains()
        
        # 1. First, handle emergency vehicles (hard constraint)
        self.create_green_waves()
        
        # 2. Handle buses to maintain schedules (hard constraint)
        self.prioritize_buses()
        
        # 3. Ensure no vehicle waits more than 3 cycles (hard constraint)
        self.handle_long_waiting_times()
        
        # 4. Ensure pedestrian crossings (hard constraint)
        self.ensure_pedestrian_crossings()
        
        # 5. Prefer green waves for longer vehicle chains (soft constraint)
        self.prefer_vehicle_chains()
        
        # 6. Optimize remaining intersections based on congestion
        self.optimize_remaining_intersections()
        
        # 7. Ensure no conflicting green lights (hard constraint)
        self.resolve_conflicts()
        
        # 8. Maintain fairness across districts (soft constraint)
        self.ensure_district_fairness()
        
        # 9. Minimize fuel consumption (soft constraint)
        self.minimize_fuel_consumption()
    
    def update_csp_variables(self):
        """Update CSP variables based on current traffic conditions"""
        for var in self.csp_variables:
            intersection_id = var['intersection_id']
            direction = var['direction']
            
            intersection = next((i for i in self.intersections if i.id == intersection_id), None)
            if not intersection:
                continue
                
            # Update priority based on queue length and wait time
            queue_length = intersection.queues[direction]
            wait_cycles = intersection.wait_cycles[direction]
            
            # Higher priority for longer queues and wait times
            priority = min(10, 1 + queue_length // 2 + wait_cycles)
            
            # Update variable
            var['priority'] = priority
            
            # Update green duration
            # Adjust green duration based on priority and congestion
            base_duration = 30
            adjustment = min(30, 5 * (priority - 1))  # 0-45 second adjustment
            
            # Longer green for higher priority
            var['green_duration'] = base_duration + adjustment
            
            # Corresponding red duration
            var['red_duration'] = max(30, 90 - adjustment)  # 30-90 second range
    
    def create_green_waves(self):
        """Create green waves for emergency vehicles"""
        for path in self.green_wave_paths:
            for i in range(len(path) - 1):
                current_id = path[i]
                next_id = path[i + 1]
                
                current = next((intersection for intersection in self.intersections 
                               if intersection.id == current_id), None)
                next_intersection = next((intersection for intersection in self.intersections 
                                        if intersection.id == next_id), None)
                
                if current and next_intersection:
                    # Determine direction from current to next
                    direction = -1
                    if next_intersection.row < current.row:
                        direction = 0  # North
                    elif next_intersection.col > current.col:
                        direction = 1  # East
                    elif next_intersection.row > current.row:
                        direction = 2  # South
                    else:
                        direction = 3  # West
                    
                    if direction != -1:
                        # Set green light in this direction
                        current.set_light(direction, 1, 60)  # Long green for emergency vehicles
                        
                        # Set red for perpendicular directions
                        perp_dirs = [(direction + 1) % 4, (direction + 3) % 4]
                        for pd in perp_dirs:
                            current.set_light(pd, 0, 60)
    
    def prioritize_buses(self):
        """Prioritize buses to maintain schedules"""
        for bus in self.buses:
            if not bus.moving and not bus.arrived and bus.behind_schedule:
                current = bus.current_intersection
                if bus.direction is not None:
                    # Set green light in bus direction
                    current.set_light(bus.direction, 1, 45)  # Longer green for buses
                    
                    # Set red for perpendicular directions
                    perp_dirs = [(bus.direction + 1) % 4, (bus.direction + 3) % 4]
                    for pd in perp_dirs:
                        current.set_light(pd, 0, 45)
    
    def handle_long_waiting_times(self):
        """Ensure no vehicle waits more than 3 cycles"""
        for intersection in self.intersections:
            for direction in range(4):
                if intersection.wait_cycles[direction] >= 3:
                    # Vehicle has waited too long, set green
                    intersection.set_light(direction, 1, 30)
                    
                    # Set red for perpendicular directions
                    perp_dirs = [(direction + 1) % 4, (direction + 3) % 4]
                    for pd in perp_dirs:
                        intersection.set_light(pd, 0, 30)
    
    def ensure_pedestrian_crossings(self):
        """Ensure pedestrian crossings are available every n seconds"""
        for intersection in self.intersections:
            # Check if pedestrian crossing is needed
            time_since_last_crossing = self.simulation_time - intersection.last_pedestrian_crossing
            
            if time_since_last_crossing > PEDESTRIAN_INTERVAL or any(intersection.pedestrian_waiting):
                # Find a direction with waiting pedestrians or choose one
                direction = -1
                
                for d in range(4):
                    if intersection.pedestrian_waiting[d]:
                        direction = d
                        break
                
                if direction == -1:
                    direction = random.randint(0, 3)
                
                # Set red for this direction to allow pedestrians to cross
                intersection.set_light(direction, 0, 15)  # 15 seconds for pedestrians
                
                # Set pedestrian timer
                intersection.pedestrian_timer = 15
                intersection.last_pedestrian_crossing = self.simulation_time
    
    def prefer_vehicle_chains(self):
        """Prefer green waves for longer vehicle chains"""
        # Sort chains by vehicle count (largest first)
        sorted_chains = sorted(self.vehicle_chains, key=lambda x: x['count'], reverse=True)
        
        for chain in sorted_chains:
            segment = chain['segment']
            current_id, next_id = segment
            
            current = next((i for i in self.intersections if i.id == current_id), None)
            next_intersection = next((i for i in self.intersections if i.id == next_id), None)
            
            if current and next_intersection:
                # Determine direction
                direction = -1
                if next_intersection.row < current.row:
                    direction = 0  # North
                elif next_intersection.col > current.col:
                    direction = 1  # East
                elif next_intersection.row > current.row:
                    direction = 2  # South
                else:
                    direction = 3  # West
                
                if direction != -1:
                    # Set green light in this direction if not already set by higher priority
                    if current.light_states[direction] != 1:
                        current.set_light(direction, 1, 40)
                        
                        # Set red for perpendicular directions
                        perp_dirs = [(direction + 1) % 4, (direction + 3) % 4]
                        for pd in perp_dirs:
                            current.set_light(pd, 0, 40)
    
    def optimize_remaining_intersections(self):
        """Optimize remaining intersections based on congestion"""
        for intersection in self.intersections:
            # Skip if already set by emergency vehicle, bus, or long wait
            if any(intersection.light_states[d] == 1 for d in range(4)):
                continue
            
            # Calculate congestion in each direction
            ns_congestion = intersection.queues[0] + intersection.queues[2]
            ew_congestion = intersection.queues[1] + intersection.queues[3]
            
            # Apply time of day factor
            if self.time_of_day == 2:  # Rush hour
                # During rush hour, prioritize main roads
                if intersection.col == 0 or intersection.col == GRID_COLS - 1:
                    # East-west edges prioritize north-south
                    ns_congestion *= 1.5
                if intersection.row == 0 or intersection.row == GRID_ROWS - 1:
                    # North-south edges prioritize east-west
                    ew_congestion *= 1.5
            
            if ns_congestion > ew_congestion:
                # Set N-S to green
                intersection.set_light(0, 1, 30 + min(30, ns_congestion * 2))
                intersection.set_light(2, 1, 30 + min(30, ns_congestion * 2))
                
                # Set E-W to red
                intersection.set_light(1, 0, 30 + min(60, ns_congestion * 2))
                intersection.set_light(3, 0, 30 + min(60, ns_congestion * 2))
            else:
                # Set E-W to green
                intersection.set_light(1, 1, 30 + min(30, ew_congestion * 2))
                intersection.set_light(3, 1, 30 + min(30, ew_congestion * 2))
                
                # Set N-S to red
                intersection.set_light(0, 0, 30 + min(60, ew_congestion * 2))
                intersection.set_light(2, 0, 30 + min(60, ew_congestion * 2))
    
    def resolve_conflicts(self):
        """Ensure no conflicting green lights"""
        for intersection in self.intersections:
            # Check for conflicts (both N-S and E-W green)
            ns_green = intersection.light_states[0] == 1 or intersection.light_states[2] == 1
            ew_green = intersection.light_states[1] == 1 or intersection.light_states[3] == 1
            
            if ns_green and ew_green:
                # Conflict detected, resolve based on queue length
                ns_queue = intersection.queues[0] + intersection.queues[2]
                ew_queue = intersection.queues[1] + intersection.queues[3]
                
                if ns_queue > ew_queue:
                    # Keep N-S green, set E-W to red
                    intersection.set_light(1, 0, intersection.red_durations[1])
                    intersection.set_light(3, 0, intersection.red_durations[3])
                else:
                    # Keep E-W green, set N-S to red
                    intersection.set_light(0, 0, intersection.red_durations[0])
                    intersection.set_light(2, 0, intersection.red_durations[2])
    
    def ensure_district_fairness(self):
        """Maintain fairness across districts"""
        # Calculate average green time per district
        district_green_times = defaultdict(list)
        
        for intersection in self.intersections:
            district = intersection.district
            avg_green_time = sum(intersection.green_time_history) / 4 if any(intersection.green_time_history) else 0
            district_green_times[district].append(avg_green_time)
        
        # Calculate average for each district
        district_averages = {}
        for district, times in district_green_times.items():
            district_averages[district] = sum(times) / len(times) if times else 0
        
        # Find districts with significantly less green time
        if district_averages:
            overall_avg = sum(district_averages.values()) / len(district_averages)
            
            for district, avg in district_averages.items():
                if avg < overall_avg * 0.8:  # District has 20% less green time
                    # Boost green time for this district
                    for intersection in self.intersections:
                        if intersection.district == district:
                            for direction in range(4):
                                if intersection.light_states[direction] == 1:
                                    # Extend green time
                                    intersection.light_timers[direction] = max(
                                        intersection.light_timers[direction],
                                        intersection.green_durations[direction] * 1.2
                                    )
    
    def minimize_fuel_consumption(self):
        """Minimize fuel consumption by reducing stop-and-go traffic"""
        # This is a simplified approach - in a real system, this would be more complex
        # We'll try to extend green lights that have vehicles approaching
        
        for intersection in self.intersections:
            for direction in range(4):
                if intersection.light_states[direction] == 1:  # Green light
                    # Check if vehicles are approaching
                    approaching_vehicles = 0
                    
                    for vehicle in self.vehicles:
                        if not vehicle.arrived and not vehicle.moving:
                            if vehicle.current_intersection.id != intersection.id:
                                continue
                                
                            if vehicle.direction == direction:
                                approaching_vehicles += 1
                    
                    if approaching_vehicles > 0:
                        # Extend green time to reduce stops
                        extension = min(15, approaching_vehicles * 3)  # Up to 15 seconds
                        intersection.light_timers[direction] = max(
                            intersection.light_timers[direction],
                            extension
                        )
    
    def update_queues(self):
        """Update queue lengths at each intersection"""
        # Reset queues
        for intersection in self.intersections:
            intersection.queues = [0, 0, 0, 0]
        
        # Count vehicles waiting at each intersection
        for vehicle in self.vehicles:
            if not vehicle.arrived and not vehicle.moving and vehicle.direction is not None:
                intersection = vehicle.current_intersection
                intersection.queues[vehicle.direction] += 1
    
    def update_congestion_levels(self):
        """Update congestion levels based on queue lengths"""
        for intersection in self.intersections:
            for direction in range(4):
                # Congestion is based on queue length and wait cycles
                intersection.congestion_levels[direction] = (
                    intersection.queues[direction] + 
                    intersection.wait_cycles[direction] * 2
                )
    
    def update_time_of_day(self):
        """Update time of day based on simulation time"""
        # Convert simulation time to hours (24-hour clock)
        hours = (self.simulation_time / 3600) % 24
        
        if 7 <= hours < 10 or 16 <= hours < 19:
            self.time_of_day = 2  # Rush hour
        elif 22 <= hours or hours < 6:
            self.time_of_day = 0  # Night
        else:
            self.time_of_day = 1  # Normal
    
    def update_weather(self):
        """Update weather conditions occasionally"""
        # Randomly change weather every ~10 minutes
        if random.random() < 0.001:  # Small chance each update
            self.weather = random.randint(0, 2)  # 0: clear, 1: rain, 2: snow
    
    def update(self, dt):
        """Update simulation state"""
        # Update simulation time
        self.simulation_time += dt
        
        # Update optimization indicator
        if hasattr(self, 'optimization_indicator') and self.optimization_indicator:
            if self.simulation_time - self.optimization_start_time > 2:  # Show for 2 seconds
                self.optimization_indicator = False
        
        # Update time of day
        self.update_time_of_day()
        
        # Update weather
        self.update_weather()
        
        # Update intersections
        for intersection in self.intersections:
            intersection.update(dt, self.simulation_time)
        
        # Update vehicles
        for vehicle in self.vehicles:
            vehicle.update(dt, self.intersections, self.simulation_time, self.weather)
        
        # Update queues and congestion
        self.update_queues()
        self.update_congestion_levels()
        
        # Run CSP optimization every 5 minutes (300 seconds)
        if self.simulation_time - self.last_optimization_time >= 300:
            self.optimize_traffic_signals()
            self.last_optimization_time = self.simulation_time
        
        # Add new vehicles occasionally
        if random.random() < 0.02:
            self.add_random_vehicle('car')
        
        # Add new buses occasionally
        if random.random() < 0.005:
            self.add_bus_with_schedule()
        
        # Add pedestrians occasionally
        if random.random() < 0.01:
            intersection = random.choice(self.intersections)
            direction = random.randint(0, 3)
            intersection.pedestrian_waiting[direction] = True
        
        # Update statistics
        self.update_statistics()
    
    def update_statistics(self):
        """Update simulation statistics"""
        # Calculate average wait time
        wait_times = [v.total_wait_time for v in self.vehicles if not v.arrived]
        self.stats['average_wait_time'] = sum(wait_times) / len(wait_times) if wait_times else 0
        
        # Calculate congestion level (0-100%)
        max_vehicles = len(self.intersections) * 4  # 4 directions per intersection
        active_vehicles = len([v for v in self.vehicles if not v.arrived])
        self.stats['congestion_level'] = (active_vehicles / max_vehicles) * 100
        
        # Calculate emergency response time
        emergency_wait_times = [v.total_wait_time for v in self.emergency_vehicles if not v.arrived]
        self.stats['emergency_response_time'] = sum(emergency_wait_times) / len(emergency_wait_times) if emergency_wait_times else 0
        
        # Calculate pedestrian wait time
        ped_wait_times = []
        for intersection in self.intersections:
            for i, waiting in enumerate(intersection.pedestrian_waiting):
                if waiting:
                    ped_wait_times.append(intersection.pedestrian_timer)
        self.stats['pedestrian_wait_time'] = sum(ped_wait_times) / len(ped_wait_times) if ped_wait_times else 0
        
        # Calculate fuel consumption
        total_fuel = sum(v.fuel_consumption for v in self.vehicles)
        self.stats['fuel_consumption'] = total_fuel
        
        # Calculate fairness index
        district_green_times = defaultdict(list)
        for intersection in self.intersections:
            district = intersection.district
            avg_green_time = sum(intersection.green_time_history) / 4 if any(intersection.green_time_history) else 0
            district_green_times[district].append(avg_green_time)
        
        district_averages = []
        for district, times in district_green_times.items():
            district_averages.append(sum(times) / len(times) if times else 0)
        
        if district_averages:
            min_avg = min(district_averages)
            max_avg = max(district_averages)
            if max_avg > 0:
                fairness = 100 * (1 - (max_avg - min_avg) / max_avg)
                self.stats['fairness_index'] = fairness
    
    def draw(self):
        """Draw the simulation"""
        # Fill screen with white
        self.screen.fill(WHITE)
        
        # Draw grid
        for i in range(GRID_ROWS + 1):
            pygame.draw.line(self.screen, GRAY, (0, i * GRID_SIZE), (GRID_COLS * GRID_SIZE, i * GRID_SIZE), 2)
        for j in range(GRID_COLS + 1):
            pygame.draw.line(self.screen, GRAY, (j * GRID_SIZE, 0), (j * GRID_SIZE, GRID_ROWS * GRID_SIZE), 2)
        
        # Draw district backgrounds
        for i in range(GRID_ROWS):
            for j in range(GRID_COLS):
                district = self.districts[i][j]
                district_colors = [(255, 255, 255), (240, 240, 255), (255, 240, 240), (240, 255, 240), (255, 255, 240)]
                if district < len(district_colors):
                    pygame.draw.rect(self.screen, district_colors[district], 
                                    (j * GRID_SIZE, i * GRID_SIZE, GRID_SIZE, GRID_SIZE), 0)
        
        # Draw roads
        for i in range(GRID_ROWS):
            for j in range(GRID_COLS):
                # Draw horizontal road
                pygame.draw.rect(self.screen, DARK_GRAY, 
                                (j * GRID_SIZE, i * GRID_SIZE + GRID_SIZE//3, 
                                 GRID_SIZE, GRID_SIZE//3))
                
                # Draw vertical road
                pygame.draw.rect(self.screen, DARK_GRAY, 
                                (j * GRID_SIZE + GRID_SIZE//3, i * GRID_SIZE, 
                                 GRID_SIZE//3, GRID_SIZE))
        
        # Draw green wave paths
        for path in self.green_wave_paths:
            points = []
            for id in path:
                intersection = next((i for i in self.intersections if i.id == id), None)
                if intersection:
                    points.append((intersection.x, intersection.y))
            
            if len(points) > 1:
                pygame.draw.lines(self.screen, GREEN, False, points, 2)
        
        # Draw road closures
        for intersection_id, direction in self.road_closures:
            intersection = next((i for i in self.intersections if i.id == intersection_id), None)
            if not intersection:
                continue
                
            # Draw X on closed road
            if direction == 0:  # North
                x1, y1 = intersection.x - 10, intersection.y - GRID_SIZE//3
                x2, y2 = intersection.x + 10, intersection.y - GRID_SIZE//6
            elif direction == 1:  # East
                x1, y1 = intersection.x + GRID_SIZE//6, intersection.y - 10
                x2, y2 = intersection.x + GRID_SIZE//3, intersection.y + 10
            elif direction == 2:  # South
                x1, y1 = intersection.x - 10, intersection.y + GRID_SIZE//6
                x2, y2 = intersection.x + 10, intersection.y + GRID_SIZE//3
            else:  # West
                x1, y1 = intersection.x - GRID_SIZE//3, intersection.y - 10
                x2, y2 = intersection.x - GRID_SIZE//6, intersection.y + 10
                
            pygame.draw.line(self.screen, BLACK, (x1, y1), (x2, y2), 3)
            pygame.draw.line(self.screen, BLACK, (x1, y2), (x2, y1), 3)
        
        # Draw intersections
        for intersection in self.intersections:
            intersection.draw(self.screen)
        
        # Draw vehicles
        for vehicle in self.vehicles:
            vehicle.draw(self.screen)
        
        # Draw statistics
        self.draw_statistics()
        
        # Draw legend
        self.draw_legend()
        
        # Draw time of day and weather
        self.draw_conditions()
        
        # Draw optimization indicator
        if hasattr(self, 'optimization_indicator') and self.optimization_indicator:
            font = pygame.font.SysFont(None, 36)
            text = font.render("CSP Optimization Running...", True, RED)
            self.screen.blit(text, (GRID_COLS * GRID_SIZE // 2 - 150, 10))
        
        # Update display
        pygame.display.flip()
    
    def draw_statistics(self):
        """Draw simulation statistics"""
        font = pygame.font.SysFont(None, 24)
        
        # Format time as hours:minutes:seconds
        hours = int(self.simulation_time // 3600) % 24
        minutes = int((self.simulation_time % 3600) // 60)
        seconds = int(self.simulation_time % 60)
        time_text = f"Simulation Time: {hours:02d}:{minutes:02d}:{seconds:02d}"
        
        # Draw statistics
        texts = [
            time_text,
            f"Average Wait Time: {self.stats['average_wait_time']:.1f}s",
            f"Congestion Level: {self.stats['congestion_level']:.1f}%",
            f"Emergency Response Time: {self.stats['emergency_response_time']:.1f}s",
            f"Pedestrian Wait Time: {self.stats['pedestrian_wait_time']:.1f}s",
            f"Fuel Consumption: {self.stats['fuel_consumption']:.1f}",
            f"Fairness Index: {self.stats['fairness_index']:.1f}%",
            f"Active Vehicles: {len([v for v in self.vehicles if not v.arrived])}"
        ]
        
        for i, text in enumerate(texts):
            surface = font.render(text, True, BLACK)
            self.screen.blit(surface, (GRID_COLS * GRID_SIZE + 20, 20 + i * 30))
    
    def draw_legend(self):
        """Draw legend"""
        font = pygame.font.SysFont(None, 24)
        
        # Legend items
        items = [
            ("Traffic Light", None),
            ("Red", RED),
            ("Green", GREEN),
            ("Yellow", YELLOW),
            ("Vehicles", None),
            ("Car", BLUE),
            ("Bus", ORANGE),
            ("Emergency", RED),
            ("Pedestrian", PURPLE),
            ("Green Wave Path", GREEN),
            ("Road Closure", BLACK)
        ]
        
        y_offset = 280
        for i, (text, color) in enumerate(items):
            if color is None:
                # Section header
                surface = font.render(text, True, BLACK)
                self.screen.blit(surface, (GRID_COLS * GRID_SIZE + 20, y_offset + i * 30))
            else:
                # Draw color box
                pygame.draw.rect(self.screen, color, 
                                (GRID_COLS * GRID_SIZE + 20, y_offset + i * 30, 15, 15))
                
                # Draw text
                surface = font.render(text, True, BLACK)
                self.screen.blit(surface, (GRID_COLS * GRID_SIZE + 40, y_offset + i * 30))
    
    def draw_conditions(self):
        """Draw time of day and weather conditions"""
        font = pygame.font.SysFont(None, 24)
        
        # Time of day
        time_labels = ["Night", "Normal", "Rush Hour"]
        time_text = f"Time of Day: {time_labels[self.time_of_day]}"
        
        # Weather
        weather_labels = ["Clear", "Rain", "Snow"]
        weather_text = f"Weather: {weather_labels[self.weather]}"
        
        # Draw texts
        time_surface = font.render(time_text, True, BLACK)
        weather_surface = font.render(weather_text, True, BLACK)
        
        self.screen.blit(time_surface, (GRID_COLS * GRID_SIZE + 20, 600))
        self.screen.blit(weather_surface, (GRID_COLS * GRID_SIZE + 20, 630))
    
    def run(self):
        """Run the simulation"""
        running = True
        paused = False
        
        while running:
            # Handle events
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_SPACE:
                        paused = not paused
                    elif event.key == pygame.K_e:
                        # Add emergency vehicle
                        start = random.choice(self.intersections)
                        end = random.choice([i for i in self.intersections if i.id != start.id])
                        self.add_emergency_vehicle(start.id, end.id)
                    elif event.key == pygame.K_b:
                        # Add bus
                        self.add_bus_with_schedule()
                    elif event.key == pygame.K_c:
                        # Add car
                        self.add_random_vehicle('car')
                    elif event.key == pygame.K_r:
                        # Add road closure
                        self.add_random_road_closure()
                    elif event.key == pygame.K_w:
                        # Change weather
                        self.weather = (self.weather + 1) % 3
                    elif event.key == pygame.K_t:
                        # Change time of day
                        self.time_of_day = (self.time_of_day + 1) % 3
            
            # Update simulation if not paused
            if not paused:
                # Calculate time step (in seconds)
                dt = 1 / ANIMATION_SPEED
                
                # Update simulation
                self.update(dt)
            
            # Draw simulation
            self.draw()
            
            # Cap the frame rate
            self.clock.tick(60)
        
        pygame.quit()

def calculate_time_complexity():
    """Calculate and print the time complexity of the CSP algorithm"""
    # Number of variables: 4 directions * 20 intersections = 80
    num_variables = 4 * 20
    
    # Domain size: possible signal timings (discretized)
    domain_size = 10  # Assuming 10 possible timing values
    
    # Worst-case time complexity for backtracking CSP: O(d^n)
    # where d is domain size and n is number of variables
    print(f"CSP Variables: {num_variables}")
    print(f"Domain Size: {domain_size}")
    print(f"Worst-case CSP complexity: O({domain_size}^{num_variables}) (exponential)")
    print(f"Heuristic approach complexity: O({num_variables}^2) = O({num_variables**2})")
    
    return {
        'num_variables': num_variables,
        'domain_size': domain_size,
        'worst_case': f"O({domain_size}^{num_variables})",
        'heuristic': f"O({num_variables}^2)"
    }

if __name__ == "__main__":
    # Calculate time complexity
    complexity = calculate_time_complexity()
    
    # Create and run traffic system
    traffic_system = TrafficSystem()
    traffic_system.run()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html
CSP Variables: 80
Domain Size: 10
Worst-case CSP complexity: O(10^80) (exponential)
Heuristic approach complexity: O(80^2) = O(6400)
